In [1]:
import pandas as pd
import numpy as np
import os
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error, mean_squared_error, mean_absolute_percentage_error, accuracy_score
import matplotlib.pyplot as plt
import seaborn as sns
import time
import random
from catboost import CatBoostClassifier, Pool

import warnings
import lightgbm as lgb
from sklearn.model_selection import KFold

In [3]:
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')

In [4]:
train

,id,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,0,15715707.0,P'eng,685.0,France,Female,31.0,4.0,152582.2,1.0,1.0,1.0,130795.52,0.0
1,1,15651022.0,Ch'ien,590.0,France,Male,43.0,2.0,0.0,2.0,1.0,0.0,96929.24,0.0
2,2,15589969.0,Yeh,588.0,France,Male,31.0,10.0,0.0,2.0,1.0,1.0,177896.92,0.0
3,3,15618661.0,Ch'ien,593.0,Spain,Male,37.0,9.0,0.0,2.0,1.0,0.0,53817.23,0.0
4,4,15771580.0,Nweke,730.0,France,Female,52.0,8.0,0.0,1.0,1.0,0.0,1276.87,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14995,14995,15700946.0,Chidubem,567.0,France,Female,29.0,6.0,0.0,2.0,1.0,0.0,69409.37,0.0
14996,14996,15652084.0,Teng,476.0,Spain,Male,29.0,3.0,0.0,2.0,1.0,0.0,116973.48,0.0
14997,14997,15657829.0,H?,773.0,France,Male,50.0,4.0,0.0,2.0,1.0,1.0,124071.71,0.0
14998,14998,15781598.0,Ch'en,648.0,France,Male,37.0,3.0,0.0,2.0,1.0,1.0,81602.02,0.0


In [5]:
test

,id,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary
0,15000,15628308.0,Nkemjika,583.0,France,Male,40.0,7.0,0.00,2.0,1.0,0.0,151481.86
1,15001,15686909.0,Chiu,595.0,France,Male,39.0,3.0,0.00,2.0,1.0,0.0,87400.53
2,15002,15579616.0,Folliero,622.0,France,Male,24.0,3.0,0.00,2.0,1.0,0.0,176712.59
3,15003,15795953.0,Chukwubuikem,577.0,Germany,Male,50.0,5.0,127305.50,1.0,1.0,1.0,28806.32
4,15004,15752731.0,Y?an,605.0,Spain,Male,39.0,2.0,88915.85,1.0,1.0,0.0,138521.02
...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,24995,15725002.0,Chukwufumnanya,695.0,France,Female,26.0,7.0,0.00,2.0,1.0,0.0,161807.84
9996,24996,15612092.0,Ijendu,754.0,Spain,Female,28.0,8.0,0.00,2.0,1.0,1.0,109626.01
9997,24997,15750055.0,Ts'ai,676.0,Germany,Male,37.0,3.0,117594.41,2.0,1.0,0.0,176692.87
9998,24998,15789563.0,Macleod,614.0,Spain,Male,38.0,2.0,0.00,2.0,1.0,0.0,124592.83


In [6]:
train['test_set'] = False
test['test_set'] = True
full_df = pd.concat([train, test], axis=0)
full_df

,id,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited,test_set
0,0,15715707.0,P'eng,685.0,France,Female,31.0,4.0,152582.20,1.0,1.0,1.0,130795.52,0.0,False
1,1,15651022.0,Ch'ien,590.0,France,Male,43.0,2.0,0.00,2.0,1.0,0.0,96929.24,0.0,False
2,2,15589969.0,Yeh,588.0,France,Male,31.0,10.0,0.00,2.0,1.0,1.0,177896.92,0.0,False
3,3,15618661.0,Ch'ien,593.0,Spain,Male,37.0,9.0,0.00,2.0,1.0,0.0,53817.23,0.0,False
4,4,15771580.0,Nweke,730.0,France,Female,52.0,8.0,0.00,1.0,1.0,0.0,1276.87,1.0,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,24995,15725002.0,Chukwufumnanya,695.0,France,Female,26.0,7.0,0.00,2.0,1.0,0.0,161807.84,NaN,True
9996,24996,15612092.0,Ijendu,754.0,Spain,Female,28.0,8.0,0.00,2.0,1.0,1.0,109626.01,NaN,True
9997,24997,15750055.0,Ts'ai,676.0,Germany,Male,37.0,3.0,117594.41,2.0,1.0,0.0,176692.87,NaN,True
9998,24998,15789563.0,Macleod,614.0,Spain,Male,38.0,2.0,0.00,2.0,1.0,0.0,124592.83,NaN,True


# EDA

In [7]:
train['IsActiveMember'].value_counts()

IsActiveMember
0.0    7656
1.0    7344
Name: count, dtype: int64

# FE

In [8]:
full_df['NumOfProducts'] = full_df['NumOfProducts'].astype(int)
full_df['IsActiveMember'] = full_df['IsActiveMember'].astype(int)
full_df['HasCrCard'] = full_df['HasCrCard'].astype(int)

In [9]:
full_df['credit_per_age'] = full_df['CreditScore'] / (full_df['Age'] + 1)
full_df['balance_to_salary'] = full_df['Balance'] / (full_df['EstimatedSalary'] + 1)
full_df['active_card'] = full_df['IsActiveMember'] * full_df['HasCrCard']

In [10]:
full_df

,id,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited,test_set,credit_per_age,balance_to_salary,active_card
0,0,15715707.0,P'eng,685.0,France,Female,31.0,4.0,152582.20,1,1,1,130795.52,0.0,False,21.406250,1.166562,1
1,1,15651022.0,Ch'ien,590.0,France,Male,43.0,2.0,0.00,2,1,0,96929.24,0.0,False,13.409091,0.000000,0
2,2,15589969.0,Yeh,588.0,France,Male,31.0,10.0,0.00,2,1,1,177896.92,0.0,False,18.375000,0.000000,1
3,3,15618661.0,Ch'ien,593.0,Spain,Male,37.0,9.0,0.00,2,1,0,53817.23,0.0,False,15.605263,0.000000,0
4,4,15771580.0,Nweke,730.0,France,Female,52.0,8.0,0.00,1,1,0,1276.87,1.0,False,13.773585,0.000000,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,24995,15725002.0,Chukwufumnanya,695.0,France,Female,26.0,7.0,0.00,2,1,0,161807.84,NaN,True,25.740741,0.000000,0
9996,24996,15612092.0,Ijendu,754.0,Spain,Female,28.0,8.0,0.00,2,1,1,109626.01,NaN,True,26.000000,0.000000,1
9997,24997,15750055.0,Ts'ai,676.0,Germany,Male,37.0,3.0,117594.41,2,1,0,176692.87,NaN,True,17.789474,0.665526,0
9998,24998,15789563.0,Macleod,614.0,Spain,Male,38.0,2.0,0.00,2,1,0,124592.83,NaN,True,15.743590,0.000000,0


# Training AutoGluon

In [11]:
drop_from_full = ['test_set', 'id', 'CustomerId']
copy_df = full_df[full_df['test_set'] == False].drop(columns=drop_from_full)

# cat_cols = copy_df.select_dtypes(include=['object', 'category', 'string']).columns.tolist()
cat_cols = ['Surname', 'Geography', 'Gender', 'NumOfProducts', 'IsActiveMember', 'HasCrCard']

In [12]:
from autogluon.tabular import TabularPredictor

# Split data into features (X) and target (y)
# x = train_df.drop('category', axis=1)
# y = train_df['category']

# # Train-test split
# # X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# # # Combine X and y for AutoGluon format
# # train_data = pd.concat([X_train, y_train], axis=1)
# # test_data = pd.concat([X_test, y_test], axis=1)


predictor = TabularPredictor(label='Exited', problem_type='binary', eval_metric='accuracy')

predictor.fit(
    copy_df,
    # num_gpus=2,
    presets='best_quality',
    time_limit=60*15,
    auto_stack=True,
    #num_bag_folds=5,
    #num_bag_sets=2,
    num_stack_levels=1,
    dynamic_stacking=True,
    # excluded_model_types=['KNN', 'FASTAI', 'NN_TORCH', 'EXTRATREES'],
)

No path specified. Models will be saved in: "AutogluonModels/ag-20250508_084059"
Verbosity: 2 (Standard Logging)
=================== System Info ===================
AutoGluon Version:  1.2
Python Version:     3.12.4
Operating System:   Linux
Platform Machine:   x86_64
Platform Version:   #56-Ubuntu SMP PREEMPT_DYNAMIC Sat Feb  8 00:37:57 UTC 2025
CPU Count:          12
Memory Avail:       10.98 GB / 15.27 GB (71.9%)
Disk Space Avail:   46.42 GB / 124.56 GB (37.3%)
Presets specified: ['best_quality']
Stack configuration (auto_stack=True): num_stack_levels=1, num_bag_folds=8, num_bag_sets=1
DyStack is enabled (dynamic_stacking=True). AutoGluon will try to determine whether the input data is affected by stacked overfitting and enable or disable stacking as a consequence.
	This is used to identify the optimal `num_stack_levels` value. Copies of AutoGluon will be fit on subsets of the data. Then holdout validation data is used to detect stacked overfitting.
	Running DyStack for up to 225s o

In [13]:
predictor.leaderboard()

,model,score_val,eval_metric,pred_time_val,fit_time,pred_time_val_marginal,fit_time_marginal,stack_level,can_infer,fit_order
0,CatBoost_r137_BAG_L1,0.906200,accuracy,0.034553,13.686233,0.034553,13.686233,1,True,18
1,WeightedEnsemble_L2,0.906200,accuracy,0.035958,14.620506,0.001405,0.934272,2,True,38
2,CatBoost_r177_BAG_L1,0.905667,accuracy,0.032806,10.906200,0.032806,10.906200,1,True,12
3,CatBoost_r69_BAG_L1,0.905667,accuracy,0.046293,15.205176,0.046293,15.205176,1,True,29
4,CatBoost_BAG_L1,0.905600,accuracy,0.037950,15.445519,0.037950,15.445519,1,True,7
5,CatBoost_r70_BAG_L1,0.905600,accuracy,0.062724,16.828328,0.062724,16.828328,1,True,33
6,CatBoost_r50_BAG_L1,0.905200,accuracy,0.049696,12.643355,0.049696,12.643355,1,True,25
7,LightGBM_BAG_L1,0.904733,accuracy,0.101177,1.810454,0.101177,1.810454,1,True,4
8,CatBoost_r167_BAG_L1,0.904600,accuracy,0.045193,6.434984,0.045193,6.434984,1,True,37
9,CatBoost_r13_BAG_L1,0.904400,accuracy,0.035566,20.930081,0.035566,20.930081,1,True,20


# Inference

In [16]:
ss = pd.read_csv('sample_submission.csv')
ss

,id,Exited
0,15000,0.5
1,15001,0.5
2,15002,0.5
3,15003,0.5
4,15004,0.5
...,...,...
9995,24995,0.5
9996,24996,0.5
9997,24997,0.5
9998,24998,0.5


In [17]:
copy_test_df = full_df[full_df['test_set'] == True].drop(columns=drop_from_full)

In [24]:
test_probs = predictor.predict_proba(copy_test_df)[1]
test_probs

0       0.036933
1       0.023516
2       0.011729
3       0.647320
4       0.135486
          ...   
9995    0.020602
9996    0.008957
9997    0.075911
9998    0.022490
9999    0.007083
Name: 1, Length: 10000, dtype: float64

In [25]:
sub_df = ss.copy()
sub_df['Exited'] = test_probs
sub_df

,id,Exited
0,15000,0.036933
1,15001,0.023516
2,15002,0.011729
3,15003,0.647320
4,15004,0.135486
...,...,...
9995,24995,0.020602
9996,24996,0.008957
9997,24997,0.075911
9998,24998,0.022490


In [26]:
sub_df.to_csv('spam_email_autogluon.csv', index=False)